In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from shapely import wkt
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score

In [2]:
train = pd.read_csv('data\Train.csv')

<>:1: SyntaxWarning: invalid escape sequence '\T'
<>:1: SyntaxWarning: invalid escape sequence '\T'
C:\Users\Aduragbemi\AppData\Local\Temp\ipykernel_23428\1100633172.py:1: SyntaxWarning: invalid escape sequence '\T'
  train = pd.read_csv('data\Train.csv')


In [3]:
train.head(3)

,FarmID,category,Crop,State,District,Sub-District,SDate,HDate,CropCoveredArea,CHeight,CNext,CLast,CTransp,IrriType,IrriSource,IrriCount,WaterCov,ExpYield,Season,geometry
0,1326576,Healthy,Paddy,Telangana,Medak,Kulcharam,2023-11-25 0:00:00,2024-04-14 0:00:00,97,54,Pea,Lentil,Transplanting,Flood,Groundwater,4,87,17,Rabi,POLYGON ((78.18143246076087 17.978884603571068...
1,1326577,Healthy,Paddy,Telangana,Medak,Kulcharam,2023-11-13 0:00:00,2024-04-26 0:00:00,82,58,Pea,Lentil,Transplanting,Flood,Canal,5,94,15,Rabi,POLYGON ((78.17545177547453 17.981074445477304...
2,1326578,Healthy,Paddy,Telangana,Medak,Kulcharam,2023-12-19 0:00:00,2024-04-28 0:00:00,92,91,Pea,Lentil,Transplanting,Flood,Canal,3,99,20,Rabi,POLYGON ((78.16914224770792 17.976212849967755...


In [4]:
test = pd.read_csv('data\Test.csv')

<>:1: SyntaxWarning: invalid escape sequence '\T'
<>:1: SyntaxWarning: invalid escape sequence '\T'
C:\Users\Aduragbemi\AppData\Local\Temp\ipykernel_23428\3210526593.py:1: SyntaxWarning: invalid escape sequence '\T'
  test = pd.read_csv('data\Test.csv')


In [5]:
test.head(3)

,FarmID,Crop,State,District,Sub-District,SDate,HDate,CropCoveredArea,CHeight,CNext,CLast,CTransp,IrriType,IrriSource,IrriCount,WaterCov,ExpYield,Season,geometry
0,85197,Paddy,Telangana,Medak,Nizampet,2023-11-12 0:00:00,2024-05-14 0:00:00,81,99,Lentil,Pea,Transplanting,Flood,Groundwater,5,81,19,Rabi,POLYGON ((78.54742059207287 18.062307326758276...
1,779677,Paddy,Telangana,Medak,Nizampet,2023-12-20 0:00:00,2024-05-28 0:00:00,91,63,Lentil,Pea,Transplanting,Flood,Canal,5,97,15,Rabi,"POLYGON ((78.54812514990353 18.06739449795583,..."
2,1331840,Paddy,Telangana,Medak,Nizampet,2023-11-29 0:00:00,2024-05-18 0:00:00,94,89,Pea,Lentil,Transplanting,Flood,Canal,5,88,15,Rabi,POLYGON ((78.56002792938853 18.079607077817514...


### **SET _FarmID_ AS THE INDEX**

In [6]:
train.set_index('FarmID', inplace=True)

In [7]:
test.set_index('FarmID', inplace=True)

### **CONVERT *SDate* AND *HDate* FEATURE TO DATETIME**

In [8]:
train['SDate'] = pd.to_datetime(train['SDate'])
train['HDate'] = pd.to_datetime(train['HDate']) 

print(train['SDate'].dtype)
print(train['HDate'].dtype)

datetime64[ns]
datetime64[ns]


In [9]:
test['SDate'] = pd.to_datetime(test['SDate'])
test['HDate'] = pd.to_datetime(test['HDate'])

print(test['SDate'].dtype)
print(test['HDate'].dtype)

datetime64[ns]
datetime64[ns]


### **EXTRACTING FEATURES FROM _geometry_**

In [10]:
train['geometry'] = train['geometry'].astype(str).apply(wkt.loads)  
train['Area'] = train['geometry'].apply(lambda x: x.area if x else None)
train['Perimeter'] = train['geometry'].apply(lambda x: x.length if x else None)   

# Calculate centroid
train['Centroid'] = train['geometry'].apply(lambda x: x.centroid if x else None)

# Optionally split centroid into latitude and longitude
train['Centroid_Lat'] = train['Centroid'].apply(lambda x: x.y if x else None)
train['Centroid_Lon'] = train['Centroid'].apply(lambda x: x.x if x else None)

train = train.drop(['geometry', 'Centroid'], axis=1)


In [11]:
test['geometry'] = test['geometry'].astype(str).apply(wkt.loads)  
test['Area'] = test['geometry'].apply(lambda x: x.area if x else None)
test['Perimeter'] = test['geometry'].apply(lambda x: x.length if x else None)   

# Calculate centroid
test['Centroid'] = test['geometry'].apply(lambda x: x.centroid if x else None)

# Optionally split centroid into latitude and longitude
test['Centroid_Lat'] = test['Centroid'].apply(lambda x: x.y if x else None)
test['Centroid_Lon'] = test['Centroid'].apply(lambda x: x.x if x else None)

test = test.drop(['geometry', 'Centroid'], axis=1)

### **EXTRACT *day*, *month* AND *week_of_month* FROM *SDate* AND *HDate***

In [12]:
from datetime import datetime

def week_of_month(dt):
    """Calculate the week of the month for a given datetime object."""
    first_day = dt.replace(day=1)
    dom = dt.day  # Day of the month
    adjusted_dom = dom + first_day.weekday()  # Adjust for the first week's weekday
    return (adjusted_dom - 1) // 7 + 1

In [13]:
train['SDay'] = train['SDate'].dt.day
train['SMonth'] = train['SDate'].dt.month
train['SWOM'] = train['SDate'].apply(week_of_month)
train['SYear'] = train['SDate'].dt.year

train = train.drop(['SDate'], axis=1)

train['HDay'] = train['HDate'].dt.day
train['HMonth'] = train['HDate'].dt.month
train['HWOM'] = train['HDate'].apply(week_of_month)
train['HYear'] = train['HDate'].dt.year

train = train.drop(['HDate'], axis=1)

train.head(3)

,category,Crop,State,District,Sub-District,CropCoveredArea,CHeight,CNext,CLast,CTransp,...,Centroid_Lat,Centroid_Lon,SDay,SMonth,SWOM,SYear,HDay,HMonth,HWOM,HYear
FarmID,,,,,,,,,,,,,,,,,,,,,
1326576,Healthy,Paddy,Telangana,Medak,Kulcharam,97,54,Pea,Lentil,Transplanting,...,17.978863,78.181592,25,11,4,2023,14,4,2,2024
1326577,Healthy,Paddy,Telangana,Medak,Kulcharam,82,58,Pea,Lentil,Transplanting,...,17.980966,78.175606,13,11,3,2023,26,4,4,2024
1326578,Healthy,Paddy,Telangana,Medak,Kulcharam,92,91,Pea,Lentil,Transplanting,...,17.976084,78.169207,19,12,4,2023,28,4,4,2024


In [14]:
test['SDay'] = test['SDate'].dt.day
test['SMonth'] = test['SDate'].dt.month
test['SWOM'] = test['SDate'].apply(week_of_month)
test['SYear'] = test['SDate'].dt.year

test = test.drop(['SDate'], axis=1)

test['HDay'] = test['HDate'].dt.day
test['HMonth'] = test['HDate'].dt.month
test['HWOM'] = test['HDate'].apply(week_of_month)
test['HYear'] = test['HDate'].dt.year

test = test.drop(['HDate'], axis=1)

test.head(3)

,Crop,State,District,Sub-District,CropCoveredArea,CHeight,CNext,CLast,CTransp,IrriType,...,Centroid_Lat,Centroid_Lon,SDay,SMonth,SWOM,SYear,HDay,HMonth,HWOM,HYear
FarmID,,,,,,,,,,,,,,,,,,,,,
85197,Paddy,Telangana,Medak,Nizampet,81,99,Lentil,Pea,Transplanting,Flood,...,18.062054,78.547474,12,11,2,2023,14,5,3,2024
779677,Paddy,Telangana,Medak,Nizampet,91,63,Lentil,Pea,Transplanting,Flood,...,18.067696,78.548407,20,12,4,2023,28,5,5,2024
1331840,Paddy,Telangana,Medak,Nizampet,94,89,Pea,Lentil,Transplanting,Flood,...,18.079480,78.560171,29,11,5,2023,18,5,3,2024


### **DROP *State* FEATURE**

In [15]:
train = train.drop(['State'], axis=1)
test = test.drop(['State'], axis=1)

In [16]:
train[['Area', 'Perimeter']].iloc[0]

Area         3.982611e-08
Perimeter    8.487694e-04
Name: 1326576, dtype: float64

### **SCALE _Area_ AND _Perimeter_**

In [17]:
train['Area'] = train['Area'] * 20000000
train['Perimeter'] = train['Perimeter'] * 2000000

In [18]:
test['Area'] = test['Area'] * 20000000
test['Perimeter'] = test['Perimeter'] * 2000000

### **CHANGE DATA TYPE OF NON-NUMERIC FEATURE TO _Category_**

In [19]:
train_cat_cols = train.select_dtypes(include='object').columns

for col in train_cat_cols:
    train[col] = train[col].astype('category')

train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7888 entries, 1326576 to 1330504
Data columns (total 27 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   category         7888 non-null   category
 1   Crop             7888 non-null   category
 2   District         7888 non-null   category
 3   Sub-District     7888 non-null   category
 4   CropCoveredArea  7888 non-null   int64   
 5   CHeight          7888 non-null   int64   
 6   CNext            7888 non-null   category
 7   CLast            7888 non-null   category
 8   CTransp          7888 non-null   category
 9   IrriType         7888 non-null   category
 10  IrriSource       7888 non-null   category
 11  IrriCount        7888 non-null   int64   
 12  WaterCov         7888 non-null   int64   
 13  ExpYield         7888 non-null   int64   
 14  Season           7888 non-null   category
 15  Area             7888 non-null   float64 
 16  Perimeter        7888 non-null   float

In [20]:
test_cat_cols = test.select_dtypes(include='object').columns

for col in test_cat_cols:
    test[col] = test[col].astype('category')

test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2718 entries, 85197 to 1326552
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   Crop             2718 non-null   category
 1   District         2718 non-null   category
 2   Sub-District     2718 non-null   category
 3   CropCoveredArea  2718 non-null   int64   
 4   CHeight          2718 non-null   int64   
 5   CNext            2718 non-null   category
 6   CLast            2718 non-null   category
 7   CTransp          2718 non-null   category
 8   IrriType         2718 non-null   category
 9   IrriSource       2718 non-null   category
 10  IrriCount        2718 non-null   int64   
 11  WaterCov         2718 non-null   int64   
 12  ExpYield         2718 non-null   int64   
 13  Season           2718 non-null   category
 14  Area             2718 non-null   float64 
 15  Perimeter        2718 non-null   float64 
 16  Centroid_Lat     2718 non-null   float64

### **ENCODE _Categorical_ FEATURES**

In [21]:
for col in train_cat_cols:
    train[col] = train[col].cat.codes

train.head(3)

,category,Crop,District,Sub-District,CropCoveredArea,CHeight,CNext,CLast,CTransp,IrriType,...,Centroid_Lat,Centroid_Lon,SDay,SMonth,SWOM,SYear,HDay,HMonth,HWOM,HYear
FarmID,,,,,,,,,,,,,,,,,,,,,
1326576,1,5,5,61,97,54,4,0,3,1,...,17.978863,78.181592,25,11,4,2023,14,4,2,2024
1326577,1,5,5,61,82,58,4,0,3,1,...,17.980966,78.175606,13,11,3,2023,26,4,4,2024
1326578,1,5,5,61,92,91,4,0,3,1,...,17.976084,78.169207,19,12,4,2023,28,4,4,2024


In [22]:
for col in test_cat_cols:
    test[col] = test[col].cat.codes

test.head(3)

,Crop,District,Sub-District,CropCoveredArea,CHeight,CNext,CLast,CTransp,IrriType,IrriSource,...,Centroid_Lat,Centroid_Lon,SDay,SMonth,SWOM,SYear,HDay,HMonth,HWOM,HYear
FarmID,,,,,,,,,,,,,,,,,,,,,
85197,5,5,25,81,99,0,4,3,1,1,...,18.062054,78.547474,12,11,2,2023,14,5,3,2024
779677,5,5,25,91,63,0,4,3,1,0,...,18.067696,78.548407,20,12,4,2023,28,5,5,2024
1331840,5,5,25,94,89,4,0,3,1,0,...,18.079480,78.560171,29,11,5,2023,18,5,3,2024


### **MODEL BUILDING**

In [23]:
X = train.drop(['category'], axis=1)
y = train['category']

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [25]:
# Oversampling the minority class
sm = SMOTE(random_state=42)
X_train, y_train = sm.fit_resample(X_train, y_train)

### **SCALING**

In [26]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### **BUILDING WITH XGBOOST**

In [27]:
model = xgb.XGBClassifier(objective='multi:softmax', 
                          booster='gbtree',
                          num_class=4, 
                          enable_categorical=True, 
                          n_estimators=500,
                          random_state=42)

In [28]:
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=True, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=500, n_jobs=None, num_class=4,
              num_parallel_tree=None, ...)

In [29]:
y_pred = model.predict(X_test)

In [30]:
f1_score(y_test, y_pred, average='weighted')

0.7417275279948529

### **BUILDING WITH CATBOOST**

In [31]:
from catboost import CatBoostClassifier

In [32]:
train['category'].value_counts()

category
1    6506
2     485
0     474
3     423
Name: count, dtype: int64

In [33]:
catmodel = CatBoostClassifier(iterations=500, 
                              learning_rate=0.3, 
                              depth=6, 
                              loss_function='MultiClass', 
                              random_state=42, 
                              class_weights=[1, 4, 4, 4])

In [34]:
catmodel.fit(X_train, y_train)

0:	learn: 1.2976054	total: 181ms	remaining: 1m 30s
1:	learn: 1.2336509	total: 205ms	remaining: 51s
2:	learn: 1.1824624	total: 224ms	remaining: 37.1s
3:	learn: 1.1457536	total: 237ms	remaining: 29.4s
4:	learn: 1.1157841	total: 250ms	remaining: 24.8s
5:	learn: 1.0932313	total: 261ms	remaining: 21.5s
6:	learn: 1.0760121	total: 271ms	remaining: 19.1s
7:	learn: 1.0506260	total: 284ms	remaining: 17.5s
8:	learn: 1.0338293	total: 297ms	remaining: 16.2s
9:	learn: 1.0201752	total: 306ms	remaining: 15s
10:	learn: 1.0030590	total: 318ms	remaining: 14.2s
11:	learn: 0.9944021	total: 333ms	remaining: 13.6s
12:	learn: 0.9800494	total: 346ms	remaining: 13s
13:	learn: 0.9664405	total: 356ms	remaining: 12.4s
14:	learn: 0.9558160	total: 368ms	remaining: 11.9s
15:	learn: 0.9443888	total: 382ms	remaining: 11.5s
16:	learn: 0.9332397	total: 399ms	remaining: 11.3s
17:	learn: 0.9204696	total: 415ms	remaining: 11.1s
18:	learn: 0.9077023	total: 433ms	remaining: 10.9s
19:	learn: 0.8993433	total: 447ms	remaining: 1

In [35]:
catpred = catmodel.predict(X_test) 

In [36]:
f1_score(y_test, catpred, average='weighted')

0.7425118596362595

### **USING LIGHTGBM**

In [37]:
import lightgbm as lgb

In [38]:
light_model = lgb.LGBMClassifier(objective='multiclass', 
                                 random_state=42, 
                                 n_estimators=500, 
                                 learning_rate=0.3)
light_model.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007303 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1721
[LightGBM] [Info] Number of data points in the train set: 18172, number of used features: 26
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294


LGBMClassifier(learning_rate=0.3, n_estimators=500, objective='multiclass',
               random_state=42)

In [39]:
light_pred = light_model.predict(X_test)

In [40]:
f1_score(y_test, light_pred, average='weighted')

0.7486159200759641

### **USING RANDOMFORESTCLASSIFIER**

In [41]:
clf = RandomForestClassifier(n_estimators=500, 
                             class_weight='balanced', 
                             random_state=42)
clf.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', n_estimators=500,
                       random_state=42)

In [42]:
clf_pred = clf.predict(X_test)

In [43]:
f1_score(y_test, clf_pred, average='weighted')

0.7439324798804751

### **PREDICTION ON TEST SET**

In [44]:
test_pred = light_model.predict(test)
test_pred

array([1, 1, 1, ..., 1, 1, 1], dtype=int8)

In [45]:
submission = pd.DataFrame(
    {
        'ID': test.index,
        'Target': test_pred
    }
)

In [46]:
category_map = {0: 'Diseased', 1: 'Healthy', 2: 'Pests', 3: 'Stressed'}

submission['Target'] = submission['Target'].map(category_map)
submission.head(3)

,ID,Target
0,85197,Healthy
1,779677,Healthy
2,1331840,Healthy


In [47]:
submission.to_csv('submission2.csv', index=False)

### **TRYING OUT OPTUNA**

In [46]:
import optuna
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

In [47]:
def objective(trial):
    # Suggest hyperparameters
    iterations = trial.suggest_int('iterations', 100, 1000)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3)
    depth = trial.suggest_int('depth', 4, 10)
    l2_leaf_reg = trial.suggest_float('l2_leaf_reg', 1e-3, 10.0)
    border_count = trial.suggest_int('border_count', 32, 255)

    # Initialize the CatBoost classifier with suggested hyperparameters
    clf = CatBoostClassifier(
        iterations=iterations,
        learning_rate=learning_rate,
        depth=depth,
        l2_leaf_reg=l2_leaf_reg,
        border_count=border_count,
        verbose=0,
        random_state=42
    )

    # Perform cross-validation
    f1_scorer = make_scorer(f1_score, average='weighted')
    scores = cross_val_score(clf, X_train, y_train, cv=3, scoring=f1_scorer)
    f1 = scores.mean()
    return f1

In [48]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=5)

[I 2025-01-20 21:12:36,097] A new study created in memory with name: no-name-2da57b68-459d-42dd-92d8-d74b7dec41e2
[I 2025-01-20 21:14:14,550] Trial 0 finished with value: 0.8908234481207624 and parameters: {'iterations': 246, 'learning_rate': 0.1222746028236668, 'depth': 9, 'l2_leaf_reg': 5.383980881617609, 'border_count': 222}. Best is trial 0 with value: 0.8908234481207624.
[I 2025-01-20 21:14:31,075] Trial 1 finished with value: 0.6945840161353458 and parameters: {'iterations': 425, 'learning_rate': 0.04816874346905372, 'depth': 4, 'l2_leaf_reg': 2.5712219004917967, 'border_count': 211}. Best is trial 0 with value: 0.8908234481207624.
[I 2025-01-20 21:14:52,409] Trial 2 finished with value: 0.8405719531958852 and parameters: {'iterations': 581, 'learning_rate': 0.173019504933549, 'depth': 4, 'l2_leaf_reg': 4.231455543381373, 'border_count': 137}. Best is trial 0 with value: 0.8908234481207624.
[I 2025-01-20 21:16:01,420] Trial 3 finished with value: 0.9227104533390906 and parameters

In [49]:
best_params = study.best_params

In [50]:
 # Train the model with the best hyperparameters
best_model = CatBoostClassifier(**best_params, verbose=0, random_state=42)
best_model.fit(X_train, y_train)

In [82]:
# Make predictions on the test data
cat_pred = best_model.predict(X_test)

In [83]:
f1_score(y_test, cat_pred, average='weighted')

0.7418915252026953

### **ENSEMBLE TECHNIQUE**

In [64]:
from sklearn.ensemble import StackingClassifier

In [66]:
# Define base models
base_models = [
    ('catboost', CatBoostClassifier(verbose=0, random_state=42)),
    ('xgboost', xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)),
    ('lightgbm', lgb.LGBMClassifier(random_state=42))
]

In [67]:
# Define the meta-model
meta_model = RandomForestClassifier(random_state=42)

In [68]:
# Define the stacking classifier
stacking_clf = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5
)

In [69]:
# Train the stacking classifier
stacking_clf.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003957 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1715
[LightGBM] [Info] Number of data points in the train set: 18172, number of used features: 24
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003244 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1714
[LightGBM] [Info] Number of data points in the train set: 14537, number of used features: 24
[LightGBM] [Info] Start training from score -1.386088
[LightGBM] [Info] Start training from score -1.386363
[LightGBM] [Info] Start training from score -1.386363
[LightGBM] [Info] Start training from score 

StackingClassifier(cv=5,
                   estimators=[('catboost',
                                <catboost.core.CatBoostClassifier object at 0x000002B0B69E4470>),
                               ('xgboost',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              device=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric='mlogloss',
                                              feature_types=Non...
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None,
                                              max_depth=None, max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=None, n_jobs=None,
                                              num_parallel_tree=None,
                                              random_state=42, ...)),
                               ('lightgbm', LGBMClassifier(random_state=42))],
                   final_estimator=RandomForestClassifier(random_state=42))

In [70]:
# Make predictions on the test data
stackpred = stacking_clf.predict(X_test)

In [71]:
f1_score(y_test, stackpred, average='weighted')

0.740308290336406